In [2]:
import numpy as np
import pandas as pd

## 1. S.I. Calculation

The seasonality indices we are going to calculate are based on the following formula:

SI for the Month = $\frac{\text{Average daily demand for month}}{\text{Average daily demand for year}}$

SI for Hour of the Day = $\frac{\text{Average hourly demand for a given hour for Mondays of a month}}{\text{Average hourly demand for Mondays of the month} * \text{Overall Monthly SI}}$

The only change we are going to make is to divide the energy demand data by 2 when we aggregrate it from half-hourly to hourly data. This is because the energy demand data is in half-hourly intervals and is in MWh units. When we aggregate it to hourly data, we need to divide it by 2 to get the hourly energy demand in MWh.

Results-


In [2]:
#WE ADD ALL THE DATA FROM YEARS INTO ONE BIG DATAFRAME
#UPDATE YOUR FILE PATH ACCORDINGLY
df = pd.read_excel(r'data\Yearly Energy Demand Data\System Demand (Actual)\2004.xlsx', index_col=0)
year = 2004
for i in range(1,19):
    temp_df = pd.read_excel("data\Yearly Energy Demand Data\System Demand (Actual)\\" + str(year + i) + '.xlsx', index_col=0)
    df = df.merge(temp_df, left_index=True, right_index=True)
df.columns = pd.to_datetime(df.columns, dayfirst=True) # converting into datetime format
df.drop(df.columns[-1], axis=1, inplace=True) # removing Jan 01 2023 column


# Monthly SI

############################################################################################################
############################################################################################################

# HALF_HOURLY DATA IS IN MWh, SO WE DIVIDE BY 2 SO THAT WE GET THE RIGHT UNITS FOR OTHER CALCULATIONS
df = df/2

############################################################################################################
############################################################################################################

hourlyall = df.copy()
dailyall = df.copy() # storing daily demand data
dailyall = dailyall.sum() # daily demand (summing up hourly demands for a day), also converting it into a pandas Series
df = dailyall.copy() 

dfdict = {}
for year in df.index.year.unique():
    dfdict[year] = []
    yearly_temp = df[df.index.year == year] # temporary yearly series
    yearly_avg = yearly_temp.mean() # average daily demand for the year
    for month in df.index.month_name().unique(): # [January,...,December]
        monthly_temp = yearly_temp[yearly_temp.index.month_name() == month] # temporay monthly series
        monthly_avg = monthly_temp.mean() # average daily demand for the month and year
        monthlysi = monthly_avg/yearly_avg
        dfdict[year].append(monthlysi)
monthlydf = pd.DataFrame(dfdict, index=df.index.month_name().unique()) # DataFrame holding S.I. for the month

monthlydf.to_excel('data/hourly-revision-data/MonthlySI.xlsx') # saving the data

overall_monthly = monthlydf.sum(axis = 1) / (2022-2004+1) # Overall S.I. for the Month

# Daily SI
# changing data from half-hourly to hourly by adding consecutive half-hour data
drop_list = []
for i in range(48):
    if i%2 == 1:
        hourlyall.iloc[i] = (hourlyall.iloc[i] + hourlyall.iloc[i-1])
    else:
        drop_list.append(hourlyall.index[i])
hourlyall.drop(drop_list,inplace=True)

dfdict = {}
for year in hourlyall.columns.year.unique():
    for month in hourlyall.columns.month_name().unique():
        for day in hourlyall.columns.day_name().unique():
            dfdict[(year,month,day)] = []
            for hour in hourlyall.index:
                # average hourly demand for a gicen day of the week, month and year (divided by S.I. of the month and day of the week)
                hourly_avg = hourlyall[hourlyall.columns[hourlyall.columns.strftime("%Y%B%A") == str(year)+month+day]].loc[hour].mean() / (overall_monthly.loc[month])
                dfdict[(year,month,day)].append(hourly_avg)
            # S.I. for the hour of the day (for particular day of the week, month and year)
            dfdict[(year,month,day)] = dfdict[(year,month,day)] / (sum(dfdict[(year,month,day)])/len(dfdict[(year,month,day)]))
hourlydf = pd.DataFrame(dfdict, index=hourlyall.index)

for year in range(2004,2023):
    year_temp = hourlydf.xs(year, axis=1, level=0)
    try:
        with pd.ExcelWriter('data/hourly-revision-data/HourlySI.xlsx', mode='a', engine='openpyxl') as writer:
            year_temp.to_excel(writer, sheet_name=str(year))
    except:
        year_temp.to_excel('data/hourly-revision-data/HourlySI.xlsx', sheet_name=str(year))

## 2. ANOVA Tests

Same as earlier, but we will use the seasonality indices calculated above to perform the ANOVA tests.
The ANOVA tests will be performed for 3 year gap pairs for aggregate data of 2 hours. 

In [4]:
import scipy.stats as stats

start_year = 2005
end_year = 2007
#hours = ['01:00', '02:00', '03:00', '04:00', '05:00', '06:00', '07:00', '08:00', '09:00', '10:00', '11:00', '12:00', '13:00', '14:00', '15:00', '16:00', '17:00', '18:00', '19:00', '20:00', '21:00', '22:00', '23:00', '00:00']
hours = ['02:00', '04:00', '06:00', '08:00', '10:00', '12:00', '14:00', '16:00', '18:00', '20:00', '22:00', '00:00']
#hours = ['03:00', '06:00', '09:00', '12:00', '15:00','18:00', '21:00', '00:00']
#hours = ['04:00', '08:00', '12:00', '16:00', '20:00', '00:00']
# types = ['weekdays', 'weekends']

while end_year < 2023:
    year_col1 = []
    year_col2 = []
    # type_col = []
    hour_col = []
    fvalue_col = []
    pvalue_col = []
    slevel_col = []
    conclusion = []
    #for daytype in types:
    for hour in hours:
        year_list = []
        for i in range(start_year,end_year+1):
            tempdf = pd.read_excel('data/hourly-revision-data/HourlySI.xlsx', sheet_name = str(i), index_col=0, header=[0,1])
            year_list.append(tempdf.loc[hour]) #.xs(daytype,axis=0,level=1))
        testdf = pd.concat(year_list, axis = 1)
        testdf.columns = [i for i in range(start_year,end_year+1)]

        fvalue, pvalue = stats.f_oneway(*[testdf[i] for i in testdf.columns])
        year_col1.append(start_year)
        year_col2.append(end_year)
        #type_col.append(daytype)
        hour_col.append(hour)
        fvalue_col.append(fvalue)
        pvalue_col.append(pvalue)
        slevel_col.append(0.05)
        if pvalue > 0.05:
            conclusion.append('Do not reject')
        else:
            conclusion.append('Reject')
            
    df = pd.DataFrame(data = {
        'Start Year' : year_col1,
        'End Year' : year_col2,
        #'Type' : type_col,
        'Hour' : hour_col,
        'f-value' : fvalue_col,
        'p-value' : pvalue_col,
        'significance level' : slevel_col,
        'Conclusion' : conclusion
        })
    if start_year == 2005:
        df.to_excel('data/hourly-revision-data/Consolidated ANOVA [2 hour blocks].xlsx', index = False, sheet_name = '{}-{}'.format(start_year, end_year), float_format = '%.3f')
    else:
        with pd.ExcelWriter('data/hourly-revision-data/Consolidated ANOVA [2 hour blocks].xlsx', mode='a', engine='openpyxl') as writer:
            df.to_excel(writer, index = False, sheet_name = '{}-{}'.format(start_year, end_year), float_format = '%.3f')
    start_year += 3
    end_year += 3

## 3. T-Tests

Same as earlier, but we will use the seasonality indices calculated above to perform the T-Tests (2 sample, one tail).

In [9]:
import pandas as pd
from scipy import stats
import math

# function that takes two arrays as inputs and performs both the t-tests and announces the result
# put the function in use in a loop for all year pairs and hours

def ttest(sample1,sample2,popmean):
    # mean of samples
    x1 = sample1.mean()
    x2 = sample2.mean()
    # number of observations
    n1 = sample1.size
    n2 = sample2.size
    # standard deviation
    s1 = sample1.std()
    s2 = sample2.std()
    # pooled standard deviation
    sp = math.sqrt(((n1-1)*(s1**2) + (n2-1)*(s2**2))/(n1+n2-2))
    # calculated for the alpha value of 0.05 and degrees of freedom (n1+n2-2)
    crit_value = stats.t.ppf(1-0.05,n1+n2-2)
    # left tailed t-test
    # popmean > 0 
    t_left = ((x1-x2)-popmean)/(sp*math.sqrt((1/n1)+(1/n2)))
    # right tailed t-test
    # popmean < 0
    t_right = ((x1-x2)-(-popmean))/(sp*math.sqrt((1/n1)+(1/n2)))
    # if t_left > -1*crit_value:
    #     print('Do not reject Null')
    # else:
    #     print('Reject Null')
    # if t_right < crit_value:
    #     print('Do not reject Null')
    # else:
    #     print('Reject Null')
    return {'left t-stastic': t_left, 'right t-statistic': t_right, 'critical value': crit_value, 'reject null': t_left < -1*crit_value and t_right > crit_value}

# VERSION 2.1- performs t-test for monthly seasonality data for all possible yearly pairs

df = pd.DataFrame(columns = [f'{i}-{j}' for i in range(2004,2022) for j in range(i+1,2023) ])

# for daytype in ['weekdays','weekends']:
tempdf = pd.read_excel(r"data/hourly-revision-data/MonthlySI.xlsx", index_col=0) # loads monthly seasonality data into a temporary dataframe for weekdays/weekends
dfdict = {} # for our dataframe (table) storing values
for i in range(2004,2022):
    for j in range(i+1,2023):
        # 2 year columns
        sample1 = tempdf[i]
        sample2 = tempdf[j]
        # procedure to calculate popmean value
        popmean = 0.00 # population mean difference
        while True:
            count = 0
            if not ttest(sample1,sample2,popmean)['reject null']:
                count +=1
    
            if count:
                #print(f'Failed- {count}, PopMean- {popmean}')
                # update popmean
                popmean = round(popmean+0.001,3)
            else:
                # print(f'FINAL RESULTS!! Failed- {count}, PopMean- {popmean}, Hour- ',hour)
                break
        # a row entry for weekdays/weekends to the dataframe
        dfdict[f'{i}-{j}'] = popmean
df.loc['si'] =  dfdict

df.transpose().to_excel('data/hourly-revision-data/Monthly S.I. t-test results.xlsx')

## 4. Models and Predictions